**【Jupyter Notebookとしての注意】**
+ 本ファイルはJupyter Notbook形式で記載するので 、(>>> や ...) といったプロンプトはプログラム例から削除しています。
+ 対話形式では、式を入れると値が表示されますが、Jupyter Notbookでは、最後の結果しか表示されないため、プログラム例に適宜「print」を追加しています。
+ print文で表示すると文字列の表現等が異なるため、プログラム例では、適宜、式に対して「repr」という関数を適用した結果をprintしています。

# 11. 標準ライブラリミニツアー – その 2
ツアーの第2部では、プロフェッショナルプログラミングを支えるもっと高度なモジュールをカバーします。ここで挙げるモジュールは、小さなスクリプトの開発ではほとんど使いません。

## 11.1. 出力のフォーマット
repr モジュールは、大きなコンテナや、深くネストしたコンテナを省略して表示するバージョンの repr() を提供しています。

In [29]:
import repr
repr.repr(set('supercalifragilisticexpialidocious'))

"set(['a', 'c', 'd', 'e', 'f', 'g', ...])"

pprint モジュールは、組み込み型やユーザ定義型をわかりやすく表示するための洗練された制御手段を提供しています。表示結果が複数行にわたる場合は、”pretty printer” と呼ばれるものが改行やインデントを追加して、データ構造がより明確になるように印字します:

In [3]:
import pprint
t = [[[['black', 'cyan'], 'white', ['green', 'red']], [['magenta',
    'yellow'], 'blue']]]
pprint.pprint(t, width=30)

[[[['black', 'cyan'],
   'white',
   ['green', 'red']],
  [['magenta', 'yellow'],
   'blue']]]


textwrap モジュールは、段落で構成された文章を、指定したスクリーン幅にぴったり収まるように調整します:

In [4]:
import textwrap
doc = """The wrap() method is just like fill() except that it returns
a list of strings instead of one big string with newlines to separate
the wrapped lines."""

print textwrap.fill(doc, width=40)

The wrap() method is just like fill()
except that it returns a list of strings
instead of one big string with newlines
to separate the wrapped lines.


locale モジュールは、文化により異なるデータ表現形式のデータベースにアクセスします。 locale の format() 関数の grouping 属性を使えば、数値を適切な桁区切り文字によりグループ化された形式に変換できます:

In [5]:
import locale
print locale.setlocale(locale.LC_ALL, 'English_United States.1252')

conv = locale.localeconv()          # get a mapping of conventions
x = 1234567.8
print locale.format("%d", x, grouping=True)

print locale.format_string("%s%.*f", (conv['currency_symbol'],
                           conv['frac_digits'], x), grouping=True)

#clean up
locale.setlocale(locale.LC_ALL, '')

English_United States.1252
1,234,567
$1,234,567.80


'Japanese_Japan.932'

## 11.2. 文字列テンプレート
string モジュールには、柔軟で、エンドユーザが簡単に編集できる簡単な構文を備えた Template クラスが入っています。このクラスを使うと、 ユーザがアプリケーションを修正することなしにアプリケーションの出力を カスタマイズできるようになります。

テンプレートでは、\$ と有効な Python 識別子名 (英数字とアンダースコア) からなるプレースホルダ名を使います。プレースホルダの周りを \{\} で囲えば、プレースホルダの後ろにスペースを挟まず、英数文字を続けることができます。\$\$ のようにすると、\$ 自体をエスケープできます:

In [6]:
from string import Template
t = Template('${village}folk send $$10 to $cause.')
print t.substitute(village='Nottingham', cause='the ditch fund')

Nottinghamfolk send $10 to the ditch fund.


substitute() メソッドは、プレースホルダに相当する値が辞書やキーワード引数にない場合に KeyError を送出します。メールマージ機能のようなアプリケーションの場合、ユーザが入力するデータは不完全なことがあるので、欠落したデータがあるとプレースホルダをそのままにして出力する safe_substitute() メソッドを使う方が適切かもしれません:

In [8]:
t = Template('Return the $item to $owner.')
d = dict(item='unladen swallow')
#t.substitute(d) # エラーから回復できないためコメントアウト
t.safe_substitute(d)

'Return the unladen swallow to $owner.'

区切り文字はデフォルトで $ ですが、Template のサブクラスを派生すると変更することができます。例えば、画像ブラウザ用に一括で名前を変更するユーティリティを作っていたとして、現在の日付や画像のシーケンス番号、ファイル形式といったプレースホルダにパーセント記号を使うことにしたら、次のようになります:

In [10]:
# 日付けが分かり難いのでjupyter化で微修正
import time, os.path
photofiles = ['img_1074.jpg', 'img_1076.jpg', 'img_1077.jpg']
class BatchRename(Template):
    delimiter = '%'
print "example of following input: Ashley_%n%f, Ashley_%d#%n%f"
fmt = raw_input('Enter rename style (%d-date %n-seqnum %f-format):  ')

t = BatchRename(fmt)
date = time.strftime('%Y-%m-%d')
for i, filename in enumerate(photofiles):
    base, ext = os.path.splitext(filename)
    newname = t.substitute(d=date, n=i, f=ext)
    print '{0} --> {1}'.format(filename, newname)

example of following input: Ashley_%n%f, Ashley_%d#%n%f
Enter rename style (%d-date %n-seqnum %f-format):  Ashley_%d#%n%f
img_1074.jpg --> Ashley_2016-06-02#0.jpg
img_1076.jpg --> Ashley_2016-06-02#1.jpg
img_1077.jpg --> Ashley_2016-06-02#2.jpg


テンプレートのもう一つの用途は、複数ある出力フォーマットからのプログラムロジックの分離です。これにより、XMLファイル用、プレーンテキストのレポート用、HTMLのwebレポート用のテンプレートに、同じプログラムロジックから値を埋め込むことができます。

## 11.3. バイナリデータレコードの操作
struct モジュールでは、様々な長さのバイナリレコード形式を操作する pack() や unpack() といった関数を提供しています。 以下の例では、 zipfile モジュールを使わずに、ZIP ファイルのヘッダ情報を 巡回する方法を示しています。"H" と "I" というパック符号は、それぞれ2バイトと4バイトの符号無し 整数を表しています。 "<" は、そのパック符号が standard サイズであり、バイトオーダーが リトルエンディアンであることを示しています:

In [11]:
import struct

data = open('myfile.zip', 'rb').read()
start = 0
for i in range(3):                      # show the first 3 file headers
    start += 14
    fields = struct.unpack('<IIIHH', data[start:start+16])
    crc32, comp_size, uncomp_size, filenamesize, extra_size = fields

    start += 16
    filename = data[start:start+filenamesize]
    start += filenamesize
    extra = data[start:start+extra_size]
    print filename, hex(crc32), comp_size, uncomp_size

    start += extra_size + comp_size     # skip to the next header

tutorial-ch10.py 0xbbf74ef9L 1631 3686
tutorial-ch3.py 0x7e4ff321 2490 7985
tutorial-ch4.py 0x51577f75 4073 10765


## 11.4. マルチスレッド
スレッド処理 (threading) とは、順序的な依存関係にない複数のタスクを分割するテクニックです。スレッドは、ユーザの入力を受け付けつつ、背後で別のタスクを動かすようなアプリケーションの応答性を高めます。同じような使用例として、I/O を別のスレッドの計算処理と並列して動作させるというものがあります。

以下のコードでは、高水準のモジュール threading でメインのプログラムを動かしながら背後で別のタスクを動作させられるようにする方法を示しています:

In [12]:
import threading, zipfile

class AsyncZip(threading.Thread):
    def __init__(self, infile, outfile):
        threading.Thread.__init__(self)
        self.infile = infile
        self.outfile = outfile
    def run(self):
        f = zipfile.ZipFile(self.outfile, 'w', zipfile.ZIP_DEFLATED)
        f.write(self.infile)
        f.close()
        print 'Finished background zip of: ', self.infile

background = AsyncZip('mydata.txt', 'myarchive.zip')
background.start()
print 'The main program continues to run in foreground.'

background.join()    # Wait for the background task to finish
print 'Main program waited until background was done.'

The main program continues to run in foreground.
Finished background zip of:  mydata.txt
Main program waited until background was done.


マルチスレッドアプリケーションを作る上で最も難しい問題は、データやリソースを共有するスレッド間の調整 (coordination)です。この問題を解決するため、threading モジュールではロックやイベント、状態変数、セマフォといった数々の同期プリミティブを提供しています。

こうしたツールは強力な一方、ちょっとした設計上の欠陥で再現困難な問題を引き起こすことがあります。したがって、タスク間調整では Queue モジュールを使って他の複数のスレッドからのリクエストを一つのスレッドに送り込み、一つのリソースへのアクセスをできるだけ一つのスレッドに集中させるほうが良いでしょう。スレッド間の通信や調整に Queue オブジェクトを使うと、設計が容易になり、可読性が高まり、信頼性が増します。

## 11.5. ログ記録
logging モジュールでは、数多くの機能をそなえた柔軟性のあるログ記録システムを提供しています。最も簡単な使い方では、ログメッセージをファイルや sys.stderr に送信します:

In [13]:
import logging
logging.debug('Debugging information')
logging.info('Informational message')
logging.warning('Warning:config file %s not found', 'server.conf')
logging.error('Error occurred')
logging.critical('Critical error -- shutting down')

これは以下の出力を生成します:

```WARNING:root:Warning:config file server.conf not found
ERROR:root:Error occurred
CRITICAL:root:Critical error -- shutting down```

デフォルトでは、info() と debug() による出力は抑制され、出力は標準エラーに送信されます。選択可能な送信先には、email、データグラム、ソケット、HTTP サーバへの送信などがあります。新たにフィルタを作成すると、DEBUG、INFO、WARNING、ERROR、CRITICAL といったメッセージのプライオリティによって異なる送信先を選択することができます。

ログ記録システムは Python から直接設定することもできますし、アプリケーションを変更しなくてもカスタマイズできるよう、ユーザが編集可能な設定ファイルによって設定することもできます。

## 11.6. 弱参照
Python は自動的にメモリを管理します (ほとんどのオブジェクトは参照カウント方式で管理し、ガベージコレクション(garbage collection)で循環参照を除去します)。オブジェクトに対する最後の参照がなくなってしばらくするとメモリは解放されます。

このようなアプローチはほとんどのアプリケーションでうまく動作しますが、中にはオブジェクトをどこか別の場所で利用している間だけ追跡しておきたい場合もあります。残念ながら、オブジェクトを追跡するだけでオブジェクトに対する恒久的な参照を作ることになってしまいます。 weakref モジュールでは、オブジェクトへの参照を作らずに追跡するためのツールを提供しています。弱参照オブジェクトが不要になると、弱参照 (weakref) テーブルから自動的に除去され、コールバック関数がトリガされます。弱参照を使う典型的な応用例には、作成コストの大きいオブジェクトのキャッシュがあります:

In [14]:
import weakref, gc
class A:
    def __init__(self, value):
        self.value = value
    def __repr__(self):
        return str(self.value)

a = A(10)                   # create a reference
d = weakref.WeakValueDictionary()
d['primary'] = a            # does not create a reference
print d['primary']          # fetch the object if it is still alive

del a                       # remove the one reference
print gc.collect()          # run garbage collection right away

print d['primary']          # entry was automatically removed

10
3


KeyError: 'primary'

## 11.7. リスト操作のためのツール
多くのデータ構造は、組み込みリスト型を使った実装で事足ります。とはいえ、時には組み込みリストとは違うパフォーマンス上のトレードオフを持つような実装が必要になこともあります。

array (配列) モジュールでは、array() オブジェクトを提供しています。配列はリストに似ていますが、同じ形式のデータだけが保存でき、よりコンパクトに保存されます。以下の例では、通常 1 要素あたり 16 バイトを必要とする Python 整数型のリストの 代りに、2 バイトの符号無しの 2 進数 (タイプコード "H") の配列を使っています:

In [15]:
from array import array
a = array('H', [4000, 10, 700, 22222])
print sum(a)

print a[1:3]

26932
array('H', [10, 700])


collections モジュールでは、deque() オブジェクトを提供しています。リスト型に似ていますが、データの追加と左端からの取り出しが速く、その一方で中間にある値の参照は遅くなります。こうしたオブジェクトはキューや幅優先のツリー探索の実装に 向いています:

In [16]:
from collections import deque
d = deque(["task1", "task2", "task3"])
print d
d.append("task4")
print "Handling", d.popleft()
print d

deque(['task1', 'task2', 'task3'])
Handling task1
deque(['task2', 'task3', 'task4'])


In [19]:
# jupyter化に当たり動くように修正
unsearched = deque([""])
def breadth_first_search(unsearched):
    while len(unsearched) != 0:
      node = unsearched.popleft()
      for m in gen_moves(node):
          if is_goal(m):
              return m
          unsearched.append(m)
    return ""

def gen_moves(node):
  return [node+"0", node+"1"]

def is_goal(g):
  return g.count("001") + g.count("010") == 4
  
breadth_first_search(unsearched)

'0010010'

リストの代わりの実装以外にも、標準ライブラリにはソート済みのリストを操作するための関数を備えた bisect のようなツールも提供しています:

In [20]:
import bisect
scores = [(100, 'perl'), (200, 'tcl'), (400, 'lua'), (500, 'python')]
bisect.insort(scores, (300, 'ruby'))
print scores

[(100, 'perl'), (200, 'tcl'), (300, 'ruby'), (400, 'lua'), (500, 'python')]


heapq モジュールは、通常のリストでヒープを実装するための関数を提供しています。ヒープでは、最も低い値をもつエントリがつねにゼロの位置に配置されます。ヒープは、毎回リストをソートすることなく、最小の値をもつ要素に繰り返しアクセスするようなアプリケーションで便利です:

In [21]:
from heapq import heapify, heappop, heappush
data = [1, 3, 5, 7, 9, 2, 4, 6, 8, 0]
heapify(data)                      # rearrange the list into heap order
heappush(data, -5)                 # add a new entry
print [heappop(data) for i in range(3)]  # fetch the three smallest entries

[-5, 0, 1]


### 11.8. 10 進浮動小数演算
decimal モジュールでは、 10 進浮動小数の算術演算をサポートする Decimal データ型を提供しています。組み込みの 2 進浮動小数の実装である float に比べて、このクラスがとりわけ便利なのは、以下の場合です

+ 財務アプリケーションやその他の正確な10進表記が必要なアプリケーション、


+ 精度の制御、


+ 法的または規制上の理由に基づく値丸めの制御、


+ 有効桁数の追跡が必要になる場合


+ ユーザが手計算の結果と同じ演算結果を期待するようなアプリケーション。

例えば、70 セントの電話代にかかる 5% の税金を計算しようとすると、10 進の浮動小数点値と 2 進の浮動小数点値では違う結果になってしまいます。計算結果を四捨五入してセント単位にしようとすると、以下のように違いがはっきり現れます:

In [22]:
from decimal import *
x = Decimal('0.70') * Decimal('1.05')
print x
print x.quantize(Decimal('0.01'))  # round to nearest cent
print round(.70 * 1.05, 2)  

0.7350
0.74
0.73


上の例で、Decimal を使った計算では、末尾桁のゼロが保存されており、有効数字2桁の被乗数から自動的に有効数字を 4 桁と判断しています。Decimal は手計算と 同じ方法で計算を行い、2 進浮動小数が 10 進小数成分を正確に表現できないことに よって起きる問題を回避しています。

Decimal クラスは厳密な値を表現できるため、2 進浮動小数点数では 期待通りに計算できないような剰余の計算や等値テストも実現できます:

In [28]:
print Decimal('1.00') % Decimal('.10')
print repr.repr(1.00 % 0.10)
print sum([Decimal('0.1')]*10) == Decimal('1.0')
print sum([0.1]*10) == 1.0

0.00
0.09999999999999995
True
False


decimal モジュールを使うと、必要なだけの精度で算術演算を行えます

In [31]:
getcontext().prec = 36
Decimal(1) / Decimal(7)

Decimal('0.142857142857142857142857142857142857')